In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [9]:
def describe(x):
    print("타입: {}".format(x.type()))
    print("크기: {}".format(x.shape))
    print("값: \n{}".format(x))
    
x_input = torch.rand(batch_size, input_dim)

## 코드 4-13 드롭아웃을 적용한 MLP

In [10]:
# 드롭아웃을 적용한 MLP
class MultilayerPerceptron(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        """
        매개변수:
            input_dim (int): 입력 벡터 크기
            hidden_dim (int): 첫 번째 Linear 층의 출력 크기
            output_dim (int): 두 번째 Linear 층의 출력 크기
        """
        super(MultilayerPerceptron, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x_in, apply_softmax=False):
        """MLP의 정방향 계산
        
        매개변수:
            x_in (torch.Tensor): 입력 데이터 텐서
                x_in.shape는 (batch, input_dim)입니다.
            apply_softmax (bool): 소프트맥스 활성화 함수를 위한 플래그
                크로스-엔트로피 손실을 사용하려면 False로 지정해야 합니다.
        반환값:
            결과 텐서. tensor.shape은 (batch, output_dim)입니다.
        """
        intermediate = F.relu(self.fc1(x_in))
        output = self.fc2(F.dropout(intermediate, p=0.5))
        
        if apply_softmax:
            output = F.softmax(output, dim=1)
        return output

batch_size = 2 # 동시에 입력될 샘플의 개수
input_dim = 3
hidden_dim = 100
output_dim = 4

# 모델 생성
mlp = MultilayerPerceptron(input_dim, hidden_dim, output_dim)
print(mlp)

y_output = mlp(x_input, apply_softmax=False)
describe(y_output)

MultilayerPerceptron(
  (fc1): Linear(in_features=3, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=4, bias=True)
)
타입: torch.FloatTensor
크기: torch.Size([2, 4])
값: 
tensor([[ 0.4132, -0.1412,  0.0511,  0.2177],
        [ 0.0562,  0.1459,  0.2482,  0.2540]], grad_fn=<AddmmBackward0>)


## 코드 4-14 인공 데이터와 Conv1d 클래스

In [11]:
# 인공 데이터와 Conv1d 클래스
batch_size = 2
one_hot_size = 10
sequence_width = 7
data = torch.randn(batch_size, one_hot_size, sequence_width)
conv1 = nn.Conv1d(in_channels=one_hot_size, out_channels=16, kernel_size=3)
intermediate1 = conv1(data)
print(data.size())
print(intermediate1.size())

torch.Size([2, 10, 7])
torch.Size([2, 16, 5])


## 코드 4-15 데이터에 반복 적용한 합성곱

In [12]:
# 데이터에 반복 적용한 합성곱
conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3)
conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3)

intermediate2 = conv2(intermediate1)
intermediate3 = conv3(intermediate2)

print(intermediate2.size())
print(intermediate3.size())

torch.Size([2, 32, 3])
torch.Size([2, 64, 1])


In [13]:
y_output = intermediate3.squeeze()
print(y_output.size())

torch.Size([2, 64])


In [14]:
intermediate2.mean(dim=0).mean(dim=1).sum()

tensor(-0.9352, grad_fn=<SumBackward0>)

## 코드 4-16 특성 벡터를 줄이는 두 가지 추가 방법

In [15]:
# 특성 벡터를 줄이는 두 가지 추가 방법

# 특성 벡터를 줄이는 방법 2
print(intermediate1.view(batch_size, -1).size())

# 특성 벡터를 줄이는 방법 2
print(torch.mean(intermediate1, dim=2).size())
# print(torch.max(intermediate1, dim=2).size())
# print(torch.sum(intermediate1, dim=2).size())

torch.Size([2, 80])
torch.Size([2, 16])
